# Handout 16
# **Sincronização com Mutex**

## **1. Introdução à Sincronização**

Em sistemas com múltiplas threads ou processos, é comum que várias unidades de execução acessem recursos compartilhados simultaneamente. Sem mecanismos adequados de controle, isso pode levar a condições de corrida (*race conditions*), onde o resultado do programa depende da ordem de execução das threads, resultando em comportamentos imprevisíveis.

**Exemplo de Condição de Corrida:**

```c
#include <pthread.h>
#include <stdio.h>

int contador = 0;

void *incrementa(void *arg) {
    for (int i = 0; i < 1000000; i++) {
        contador++;
    }
    return NULL;
}

int main() {
    pthread_t t1, t2;
    pthread_create(&t1, NULL, incrementa, NULL);
    pthread_create(&t2, NULL, incrementa, NULL);
    pthread_join(t1, NULL);
    pthread_join(t2, NULL);
    printf("Contador final: %d\n", contador);
    return 0;
}

```

**Possível Resultado:**

- O valor esperado para contador seria 2.000.000, mas devido à falta de sincronização, o resultado pode ser menor, indicando uma condição de corrida.

## **2. Mutex: Exclusão Mútua**

Um *mutex* (do inglês *mutual exclusion*) é um mecanismo de sincronização que permite que apenas uma thread acesse um recurso compartilhado por vez, prevenindo condições de corrida.

### **2.1. Operações Básicas com Mutex**

- **Inicialização:** Antes de usar um mutex, é necessário inicializá-lo.
- **Lock (Travar):** Uma thread que deseja acessar um recurso protegido deve adquirir o mutex.
- **Unlock (Destravar):** Após acessar o recurso, a thread deve liberar o mutex.
- **Destruição:** Quando o mutex não for mais necessário, ele deve ser destruído para liberar recursos.

### **2.2. Funções Principais**

- **`pthread_mutex_init`**: Inicializa um mutex.
- **`pthread_mutex_lock`**: Adquire o mutex (bloqueia se já estiver em uso).
- **`pthread_mutex_unlock`**: Libera o mutex.
- **`pthread_mutex_destroy`**: Destroi o mutex.

---

## **3. Exemplo Prático: Uso de Mutex**

**Objetivo:** Corrigir o exemplo anterior para evitar a condição de corrida utilizando mutex.



In [ ]:
#include <pthread.h>
#include <stdio.h>

int contador = 0;
pthread_mutex_t mutex_contador;

void *incrementa(void *arg) {
    for (int i = 0; i < 1000000; i++) {
        pthread_mutex_lock(&mutex_contador);
        contador++;
        pthread_mutex_unlock(&mutex_contador);
    }
    return NULL;
}

int main() {
    pthread_t t1, t2;
    pthread_mutex_init(&mutex_contador, NULL);
    pthread_create(&t1, NULL, incrementa, NULL);
    pthread_create(&t2, NULL, incrementa, NULL);
    pthread_join(t1, NULL);
    pthread_join(t2, NULL);
    printf("Contador final: %d\n", contador);
    pthread_mutex_destroy(&mutex_contador);
    return 0;
}

**Explicação:**

- Inicialização: pthread_mutex_init inicializa o mutex antes de criar as threads.
- Uso do Mutex: Dentro da função incrementa, pthread_mutex_lock é chamado antes de incrementar contador, garantindo que apenas uma thread por vez execute essa seção crítica. Após incrementar, pthread_mutex_unlock libera o mutex.
- Destruição: Após o uso, pthread_mutex_destroy libera os recursos associados ao mutex.

**Resultado Esperado:**

- O valor de contador será consistentemente 2.000.000, indicando que a condição de corrida foi eliminada.

## **4. Considerações sobre o Uso de Mutex**

- **Desempenho:** O uso excessivo de mutexes pode levar a contenção, onde múltiplas threads competem pelo mesmo recurso, reduzindo o desempenho. É importante proteger apenas as seções críticas do código.
- **Deadlocks:** Se não utilizados corretamente, mutexes podem levar a deadlocks, onde duas ou mais threads ficam bloqueadas indefinidamente esperando por recursos.

### **Dicas para evitar Deadlocks**:
- **Ordem Consistente:** Sempre adquira múltiplos mutexes na mesma ordem.
- **Minimize o Tempo de Bloqueio:** Evite manter um mutex bloqueado por longos períodos.
- **Timeouts:** Considere usar mecanismos de tempo limite ao tentar adquirir um mutex.

---

## **5. Alternativas aos Mutexes**

Além dos mutexes, existem outros mecanismos de sincronização que podem ser mais adequados dependendo do cenário:

- **Semáforos:** Controlam o acesso a múltiplas instâncias de um recurso compartilhado.  
  **Exemplo de uso:** Controle de um pool de conexões.

- **Variáveis de Condição:** Permitem que threads esperem até que uma condição específica seja atendida.  
  **Exemplo de uso:** Produtores e consumidores sincronizando a produção e o consumo de itens em um buffer.

Esses mecanismos são frequentemente utilizados em conjunto para resolver diferentes tipos de problemas de concorrência.
